<a href="https://colab.research.google.com/github/s528661/nlp100/blob/JDLA_Engeneer_20240503/NLP100Knock_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第6章: 機械学習  
本章では，Fabio Gasparetti氏が公開しているNews Aggregator Data Setを用い，ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」のカテゴリに分類するタスク（カテゴリ分類）に取り組む．  

In [2]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.6 MB/s eta 0:00:00


In [3]:
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

# import dask.array as da

import lightgbm as lgb

# Neural Language Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import RidgeClassifier
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## 50. データの入手・整形
News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．  

1. ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．  
2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．  
3. 抽出された事例をランダムに並び替える．  
4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．  
5. 学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．





In [4]:
# 1. ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate
!unzip ./NewsAggregatorDataset.zip

--2024-05-03 03:36:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas     [    <=>             ]  27.87M  35.0MB/s    in 0.8s    

2024-05-03 03:36:37 (35.0 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [7]:
# 2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
data0 = pd.read_csv('newsCorpora.csv',sep='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])
data = data0[data0['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [11]:
# 3. 抽出された事例をランダムに並び替える．
# 4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．
train0, test = train_test_split(data, test_size=0.1, shuffle=True)
train, val = train_test_split(train0, test_size=0.1, shuffle=True)

In [12]:
# 4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．
train.to_csv("train.txt", sep="\t", index=False, header=None)
val.to_csv("valid.txt", sep="\t", index=False, header=None)
test.to_csv("test.txt", sep="\t", index=False, header=None)

In [16]:
# 5. 学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．
print("train:{}\nvalid:{}\ntest:{}".format(train.shape, val.shape, test.shape))

train:(10805, 8)
valid:(1201, 8)
test:(1334, 8)


## 51. 特徴量抽出  
学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ．   
なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．  

In [17]:
# 前処理（大文字をすべて小文字にする、数字を削除）
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [18]:
# データを特徴量に変換
## TITLE を小文字にする
data['TITLE'] = data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-18-a49b5be817da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TITLE'] = data['TITLE'].apply(lambda x: preprocessing(x))


In [24]:
## 1-gram, 2-gramでTfidfを計算
vectorizer = TfidfVectorizer(min_df=10, ngram_range=(1, 2))

X = vectorizer.fit_transform(data['TITLE']).toarray()
X = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())

In [25]:
X_train0, X_test = train_test_split(X, test_size=0.1, shuffle=True)
X_train, X_val = train_test_split(X_train0, test_size=0.1, shuffle=True)

In [26]:
X_train.to_csv("train.feature.txt", sep="\t", index=False, header=None)
X_val.to_csv("valid.feature.txt", sep="\t", index=False, header=None)
X_test.to_csv("test.feature.txt", sep="\t", index=False, header=None)

## 52. 学習
51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

In [27]:
y_train = data.iloc[X_train.index]['CATEGORY']
y_val = data.iloc[X_val.index]['CATEGORY']
y_test = data.iloc[X_test.index]['CATEGORY']

ロジスティック回帰  
- 予測値を 0 ~ 1 に変換  
- 確率が高いカテゴリを選択  
  
$\sigma(x) = \frac{1}{1 + \exp^{-x}}$

In [28]:
model = LogisticRegression(random_state=64, max_iter=10000)
model.fit(X_train,y_train)

LogisticRegression(max_iter=10000, random_state=64)

## 53. 予測  
52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．  

In [29]:
y_val_pred = model.predict(X_val)
y_val_pred_proba = model.predict_proba(X_val)

In [30]:
y_val_pred

array(['b', 'e', 'e', ..., 'e', 'b', 'e'], dtype=object)

In [31]:
y_val_pred_proba

array([[0.94423657, 0.02483694, 0.0114473 , 0.01947919],
       [0.03840239, 0.89660762, 0.03912327, 0.02586672],
       [0.05042273, 0.91644765, 0.01617553, 0.01695409],
       ...,
       [0.37501741, 0.46415084, 0.069691  , 0.09114075],
       [0.89938979, 0.02617146, 0.01566621, 0.05877255],
       [0.04743907, 0.90500125, 0.0175499 , 0.03000978]])